In [10]:
# print version / should be 1.6.x
println(versioninfo())

# load centralized paths dictionary
using YAML
PATHS_FILE = "../../../PATHS.YML"
PATHS = YAML.load_file(PATHS_FILE);

"""
    load_path(keys::Vector{String}):String

Return the absolute path for a requested item within the nested PATHS dictionary.
"""
function loadpath(keys::Vector{String}):String
    # recusively assemble paths from keys
    requested_path = foldl((x, y) -> getindex(x, y), keys; init=PATHS)
    # rephrase abs path from relative paths to be platform independent.
    realpath(joinpath(splitdir(realpath(PATHS_FILE))[1], requested_path))
end;

Julia Version 1.6.6
Commit b8708f954a (2022-03-28 07:17 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i5-4260U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
nothing


In [14]:
using CSV
using DataFrames

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1342


In [13]:
# we load the project path as it is used by CliMA Land v01 papers
using Pkg
PROJECT_PATH = loadpath(["EXTERNAL", "01_CLIMA_LAND_V01", "ROOT"])
Pkg.activate(PROJECT_PATH)


  Activating environment at `~/Desktop/island_in_the_sun/python/2022_04_lsm/ext_src/01_julia_land_v01/Land/Project.toml`


# First exploration of CliMA Land v0.1

For our emulator, we want to use the CliMA Land model (v0.1), as it has [some advantages](../../../docs/reporting/01_meetings_documentation.md#M05_climaland). Therefore, it is important to understand the model. Thus, we will install it and follow/disentangle the code of the trial. Later, we can change some parameters and see the response.

## Installation

I installed CliMA Land v0.1 following the instructions of its [README](https://github.com/CliMA/Land/blob/41f90baa90c5b8e761fdcce5005eebde2f42b26b/README.md).

The script `clima-land.jl` was stored in the cloned repository, the dataset`era5_2019_117_296_1X.csv` in our cache directory.

## Disentangling the wrapper

The wrapper of the tutorial should allow to run the LSM simulations for single points. To reproduce and alternate it, I will go through the code subsequently.

**This is the whole script:**

In [7]:
# load wrapper path and print its code
CL_WRAPPER_FILE = loadpath(["EXTERNAL", "01_CLIMA_LAND_V01", "WRAPPER"]);

open(CL_WRAPPER_FILE) do file
    println(read(file, String))
end

using LazyArtifacts

using CanopyLayers: BLUE, EVI, FourBandsFittingHybrid, NDVI, NIR, NIRv, NIRvES, RED, SIF_740, SIF_WL, SIF_fluxes!, canopy_fluxes!, canopy_geometry!, canopy_matrices!, fit_soil_mat!, short_wave!, soil_albedos
using DataFrames: DataFrame
using GriddingMachine: Indexer.lat_ind, Indexer.lon_ind
using Photosynthesis: AbstractPhotoModelParaSet, AirLayer, leaf_ETR!
using PkgUtility: numerical∫, read_csv, save_csv!
using PlantHydraulics: soil_p_25_swc, temperature_effects!
using SoilPlantAirContinuum: SPACMono, initialize_spac_canopy!, update_Cab!, update_LAI!, zenith_angle
using Statistics: mean
using StomataModels: BetaGLinearPsoil, CanopyLayer, ESMMedlyn, EmpiricalStomatalModel, GswDrive, gas_exchange!, gsw_control!, stomatal_conductance, update_leaf_TP!, β_factor
using UnPack: @unpack
using WaterPhysics: saturation_vapor_pressure


# function to create a SPAC node
create_spac(lat::FT, lon::FT) where {FT<:AbstractFloat} = (
    _soil_bnds = FT[0, -0.1, -0.35, -1, -3];
 

### Function per function

I will go through the functions one after the other:

#### `create_spac`

The CliMA Land model uses instances of soil plant air continua (a.k.a SPACs). The [SPACMono](https://yujie-w.github.io/SoilPlantAirContinuum.jl/stable/API/#SoilPlantAirContinuum.SPACMono) is the atomic 1D abstraction of a grid cell. It represents a single species of plant interacting with soil and atmosphere and has properties like:
- Root depth
- Canopy maximal height
- Longitude and Latitude
- Ground area
- Layers of Roots/Canopy
- Canopy water flux per ground area
- ...

The variable `soil_bnds` represents the discrete boundaries of soil layers (here 4 as used in [Wang, et al. (2022)](#wangModelingGlobalCarbon2022a), like in JULES) for different hydraulic parameters. It is used for soil hydraulic conductivity by [van Genuchten (1980)](#vangenuchtenClosedformEquationPredicting1980). One could use 6 or the 10 CLM layers published globally by [Dai, et al. (2019)](#daiGlobalHighResolutionData2019).

In short, this function builds an instance for a single model grid cell.

#### `query_data`

The trial uses a dataset for 26.5°N, 115.5°E in 2019. Its data is stored in `era5_2019_117_296_1X.csv`. This function helps to translate the temporal and spatial coordinates to the filename of the corresponding data and instantiates a SPAC for it.

#### `update_gsw!`

This function updates the stomatal conductance (instantiated within `CanopyLayer` object) for a timestep. Therefore, it uses a set of photosynthesis parameter (C3 or C4 dependent), the surrounding air conditions of the canopy (`AirLayer`), temperature difference, time interval and a correction factor.

#### `clima_land!`

Within the wrapper script, the core function is the `clima_land!` inplace simulation. It works on a `SPACMono` instance and stores the model outputs within a dataframe.

The function uses the input datasheet and iterates through each row (and days as float/FDOY)

In [20]:
CL_WRAPPER_CSVFILE = loadpath(["DATA", "TRIAL", "CLIMA_LAND", "WRAPPER_DATA"]);

df = CSV.read(CL_WRAPPER_CSVFILE, DataFrame);

In [54]:
withenv("COLUMNS" => 1000) do
    display(df[1:10,:])
end

,u10,v10,d2m,t2m,evavt,msdrswrf,msdwswrf,skt,stl1,stl2,stl3,stl4,sp,swvl1,swvl2,swvl3,swvl4,FDOY,WIND,RAD_DIF,RAD_DIR,VPD
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.244298,-2.68466,273.025,274.239,-4.163e-5,2.24389,10.2953,273.663,275.887,277.731,284.404,291.136,1.00036e5,0.458745,0.459198,0.450892,0.442369,0.341667,2.69576,8.05141,2.24389,55.615
2,-0.292182,-2.59695,273.003,274.174,-1.6108e-6,7.27977,58.3956,274.384,275.535,277.703,284.38,291.13,1.00046e5,0.458485,0.45914,0.450883,0.442373,0.383333,2.61333,51.1159,7.27977,53.4656
3,-0.213711,-2.68381,273.536,275.396,-3.09868e-6,46.4403,150.369,274.959,276.401,277.704,284.355,291.125,1.00085e5,0.458202,0.459061,0.450875,0.442372,0.425,2.6923,103.928,46.4403,89.8877
4,-0.0344099,-2.88229,273.571,275.398,-4.46663e-6,60.6056,211.243,275.651,276.233,277.742,284.331,291.12,99987.0,0.457904,0.458969,0.450863,0.442373,0.466667,2.8825,150.638,60.6056,88.3688
5,0.114259,-3.13434,273.468,275.289,-5.80835e-6,66.2457,229.438,276.802,276.265,277.813,284.307,291.114,99857.1,0.457579,0.458837,0.45084,0.442376,0.508333,3.13642,163.193,66.2457,87.4814
6,0.219577,-3.23774,273.78,276.141,-7.11259e-6,64.8182,211.77,276.571,277.356,277.909,284.284,291.109,99709.5,0.457293,0.458717,0.450824,0.44238,0.55,3.24518,146.952,64.8182,117.727
7,0.249901,-3.21085,273.758,276.263,-8.37186e-6,131.052,272.068,277.014,277.865,278.026,284.262,291.104,99656.8,0.457048,0.458578,0.450802,0.442381,0.591667,3.22056,141.017,131.052,125.305
8,0.355791,-3.26196,273.571,276.075,-9.75855e-6,14.2745,104.613,277.08,278.021,278.148,284.24,291.098,99645.7,0.456806,0.458446,0.450779,0.442387,0.633333,3.28131,90.339,14.2745,123.83
9,0.437916,-3.35342,273.611,275.675,-1.1134e-5,6.52928,75.756,276.293,277.816,278.254,284.219,291.093,99666.8,0.456684,0.458345,0.450763,0.442385,0.675,3.38189,69.2267,6.52928,100.821


The used variables are:

---
# References
[**Dai Y, Xin Q, Wei N, Zhang Y, Shangguan W, Yuan H, Zhang S, Liu S, Lu X. 2019.**](https://doi.org/10.1029/2019MS001784) <a id="daiGlobalHighResolutionData2019"></a> A Global High-Resolution Data Set of Soil Hydraulic and Thermal Properties for Land Surface Modeling. Journal of Advances in Modeling Earth Systems 11: 2996–3023.

[**van Genuchten MTh. 1980.**](https://doi.org/10.2136/sssaj1980.03615995004400050002x) <a id="vangenuchtenClosedformEquationPredicting1980"></a> A Closed-form Equation for Predicting the Hydraulic Conductivity of Unsaturated Soils. Soil Science Society of America Journal 44: 892–898.

[**Wang Y, Braghiere RK, Longo M, Norton A, Köhler P, Doughty R, Yin Y, Bloom AA, Frankenberg C. 2022.**](https://doi.org/10.1002/essoar.10509956.1) <a id="wangModelingGlobalCarbon2022a"></a> Modeling global carbon and water fluxes and hyperspectral canopy radiative transfer simultaneously using a next generation land surface model—CliMA Land. Biological Sciences.
